## 使用__slots__


我们定义一个class，创建一个实例后，我们可以给实例绑定任何属性和方法，这就是动态语言的灵活性。

In [1]:
class Student(object):
    pass

In [2]:
s = Student()
s.name = 'Sam'
print(s.name)

Sam


In [3]:
def set_age(self, age):
    self.age = age

In [4]:
from types import MethodType

s.set_age = MethodType(set_age, s)

In [5]:
s.set_age(25)

In [7]:
s.age

25

如果我们想限制实例的属性怎么办？比如只允许对Student实例添加name, age属性。

为了达到目的，python允许在定义class的时候，定义一个特殊的__slots__变量，来限制class 实例能添加的属性。

In [8]:
class Student(object):
    __slots__ = ('name', 'age')  # 用tuple定义允许绑定的属性名称
    

In [12]:
s = Student()
s.name = 'Susan'
s.age = 25
s.score = 99  #报错，因为__slots__函数的限制，只能绑定name and age

AttributeError: 'Student' object has no attribute 'score'

# @property


前面提到过装饰器(decorator)，这个是python内置的装饰器，很好用。负责把一个方法变成属性调用：

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作：

In [19]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer')
        if value < 0 or value > 100:
            raise ValueError('score must between 0~100')
        self._score = value

In [20]:
s = Student()
s.score = 60

In [21]:
s.score

60

注意到这个神奇的@property，我们在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现的。

还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性：

In [24]:
# 另一个例子
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth

上面的birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。

@property广泛应用在类的定义中，可以让调用者写出简短的代码，同时保证对参数进行必要的检查，这样，程序运行时就减少了出错的可能性。

# 多重继承

其实就是拓扑结构，核心思路就是一个子类可以继承多个父类

In [25]:
# 动物的树图
class Animal(object):
    pass

# 大类:
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

# 各种动物:
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass


In [27]:
# 飞行和爬行类的树图
class Runnable(object):
    def run(self):
        print('Running...')

class Flyable(object):
    def fly(self):
        print('Flying...')


In [28]:
# 同时继承哺乳动物和爬行动物
class Dog(Mammal, Runnable):
    pass

## MixIn

顾名思义，通过多重继承实现的设计

In [29]:
class Dog(Mammal, RunnableMixIn, CarnivorousMixIn):
    pass


NameError: name 'RunnableMixIn' is not defined

# 定制类

- __str__

In [1]:
# 先定义一个Student类，打印一个实例：
class Student(object):
    def __init__(self, name):
        self.name = name
        
print(Student('Sam'))

可以看到，打印出来的实例是一个object，不好看

In [6]:
# ，下面在类中添加__str__：
class Student1(object):
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return 'Student object (name: %s)' % self.name

In [7]:
print(Student1('Amy'))

Student object (name: Amy)


这是因为直接显示变量调用的不是__str__()，而是__repr__()，两者的区别是__str__()返回用户看到的字符串，而__repr__()返回程序开发者看到的字符串，也就是说，__repr__()是为调试服务的。

解决办法是再定义一个__repr__()。但是通常__str__()和__repr__()代码都是一样的，所以，有个偷懒的写法：

In [8]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__

- __iter__

In [10]:
"""如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个__iter__()方法，该方法返回一个迭代对象，
然后，Python的for循环就会不断调用该迭代对象的__next__()方法拿到循环的下一个值，
直到遇到StopIteration错误时退出循环。
我们以斐波那契数列为例，写一个Fib类，可以作用于for循环："""

class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.a, self.b = self.b, self.a+self.b
        if self.a > 1000:
            raise StopIteration()
        return self.a

In [12]:
for i in Fib():
    print(i)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987


- getitem

按下标取出元素，需要用到__getitem__()

In [13]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a+ b
        return a

In [14]:
f = Fib()
f[1]

1

注意这时还无法切片，一旦切片会报错，原因是传入的参数可能是一个int，也可能是一个切片对象，所以，要作出判断：

In [15]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a+b
            return a
        if isinstance(n,slice): # n是切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x > start:
                    L.append(a)
                a, b = b, a+b
            return L

In [17]:
f = Fib()
f[0:5]

[1, 2, 3, 5]